In [31]:
import tkinter as tk
from tkinter import messagebox, ttk
import json
from datetime import date, datetime
from docx import Document
import os
import logging
import sqlite3
import uuid

# Configure logging for debugging
logging.basicConfig(filename='physio_app.log', level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

# Date string to date object conversion
def str_to_date(obj):
    if isinstance(obj, str):
        try:
            return datetime.strptime(obj, '%Y-%m-%d').date()
        except ValueError:
            return obj
    return obj

print("Cell 1: Imports and Utilities executed")

Cell 1: Imports and Utilities executed


In [32]:
#Cell 2 revised at 15:48
# Cell 2
from datetime import datetime

def str_to_date(date_str):
    if isinstance(date_str, datetime.date):
        return date_str
    if not date_str or date_str == 'None':
        return None
    return datetime.strptime(date_str, '%Y-%m-%d').date()

class Patient:
    def __init__(self, first_name, surname, contact, dob, gender, medical_aid, medical_aid_number, occupation,
                 referral_diagnosis, physiotherapist_name, medical_history, diagnosis):
        self.first_name = first_name
        self.surname = surname
        self.contact = contact
        self.dob = dob
        self.gender = gender
        self.medical_aid = medical_aid
        self.medical_aid_number = medical_aid_number
        self.occupation = occupation
        self.referral_diagnosis = referral_diagnosis
        self.physiotherapist_name = physiotherapist_name
        self.medical_history = medical_history
        self.diagnosis = diagnosis
        self.assessments = []
        self.reassessments = []

    def add_assessment(self, assessment):
        self.assessments.append(assessment)

    def add_reassessment(self, reassessment):
        self.reassessments.append(reassessment)

class MedicalHistory:
    def __init__(self, conditions, surgeries, allergies):
        self.conditions = conditions
        self.surgeries = surgeries
        self.allergies = allergies

class Diagnosis:
    def __init__(self, diagnosis, icd_code):
        self.diagnosis = diagnosis
        self.icd_code = icd_code

class TreatmentPlan:
    def __init__(self, exercises, therapies, modalities, interventions, home_exercise_program):
        self.exercises = exercises
        self.therapies = therapies
        self.modalities = modalities
        self.interventions = interventions
        self.home_exercise_program = home_exercise_program

class Assessment:
    def __init__(self, findings, assessment_date, subjective_symptoms, objective_findings, history, treatment_notes):
        self.findings = findings
        self.assessment_date = assessment_date
        self.subjective_symptoms = subjective_symptoms
        self.objective_findings = objective_findings
        self.history = history
        self.treatment_notes = treatment_notes
        self.treatment_plan = None  # To be set when adding treatment plan

    def to_dict(self):
        return {
            'findings': self.findings,
            'assessment_date': str(self.assessment_date),
            'subjective_symptoms': self.subjective_symptoms,
            'objective_findings': self.objective_findings,
            'history': self.history,
            'treatment_notes': self.treatment_notes,
            'treatment_plan': {
                'exercises': self.treatment_plan.exercises,
                'therapies': self.treatment_plan.therapies,
                'modalities': self.treatment_plan.modalities,
                'interventions': self.treatment_plan.interventions,
                'home_exercise_program': self.treatment_plan.home_exercise_program
            } if self.treatment_plan else None
        }

class Reassessment:
    def __init__(self, reassessment_date, progress_notes, objective_findings, recommendations):
        self.reassessment_date = reassessment_date
        self.progress_notes = progress_notes
        self.objective_findings = objective_findings
        self.recommendations = recommendations
        self.treatment_plan = None  # To be set when adding treatment plan

    def to_dict(self):
        return {
            'reassessment_date': str(self.reassessment_date),
            'progress_notes': self.progress_notes,
            'objective_findings': self.objective_findings,
            'recommendations': self.recommendations,
            'treatment_plan': {
                'exercises': self.treatment_plan.exercises,
                'therapies': self.treatment_plan.therapies,
                'modalities': self.treatment_plan.modalities,
                'interventions': self.treatment_plan.interventions,
                'home_exercise_program': self.treatment_plan.home_exercise_program
            } if self.treatment_plan else None
        }

print("Cell 2: Patient, MedicalHistory, Diagnosis, TreatmentPlan, Assessment, and Reassessment classes defined")

Cell 2: Patient, MedicalHistory, Diagnosis, TreatmentPlan, Assessment, and Reassessment classes defined


In [ ]:
#Cell 3 Revised at 19:30
# Cell 3
import sqlite3
from datetime import datetime
import tkinter as tk
from tkinter import ttk, messagebox
import logging
import uuid
import json
from docx import Document

# Register custom date adapter and converter
def adapt_date(date):
    return date.strftime('%Y-%m-%d')

def convert_date(date_str):
    return datetime.strptime(date_str.decode('utf-8'), '%Y-%m-%d').date()

sqlite3.register_adapter(datetime.date, adapt_date)
sqlite3.register_converter('DATE', convert_date)

class PhysioApp:
    def __init__(self):
        print("PhysioApp: Initializing")
        self.root = tk.Tk()
        self.root.title('Patient Management System')
        # Set window size and center it
        window_width = 600
        window_height = 400
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        x = (screen_width - window_width) // 2
        y = (screen_height - window_height) // 2
        self.root.geometry(f'{window_width}x{window_height}+{x}+{y}')
        self.root.minsize(600, 400)
        self.patients = {}
        self.current_patient = None
        self.patient_counter = 1
        self.frames = {}
        self.conn = None  # Initialize as None
        self.cursor = None
        self.ensure_connection()  # Open connection
        self.initialize_database()  # Initialize database and tables
        self.load_patients()
        self.create_main_menu()
        self.edit_patient_var = tk.StringVar()
        self.delete_patient_var = tk.StringVar()
        print("PhysioApp: Initialization complete")
        # Handle window close
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def ensure_connection(self):
        """Ensure the database connection is open."""
        if self.conn is None:
            print("PhysioApp: Opening database connection")
            self.conn = sqlite3.connect('physio.db', detect_types=sqlite3.PARSE_DECLTYPES)
            self.cursor = self.conn.cursor()
        return self.conn

    def close_connection(self):
        """Close the database connection if open."""
        if self.conn is not None:
            print("PhysioApp: Closing database connection")
            self.conn.close()
            self.conn = None
            self.cursor = None

    def on_closing(self):
        """Handle window close event."""
        if messagebox.askokcancel("Quit", "Do you want to quit?"):
            self.close_connection()
            self.root.destroy()

    def initialize_database(self):
        """Initialize the database, creating tables only if necessary."""
        print("PhysioApp: Initializing database")
        self.ensure_connection()
        # Check if Patients table exists
        self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='Patients'")
        table_exists = self.cursor.fetchone()

        if not table_exists:
            print("PhysioApp: Creating tables (no existing Patients table)")
            self.create_tables()
        else:
            # Verify schema
            self.cursor.execute("PRAGMA table_info(Patients)")
            columns = [info[1] for info in self.cursor.fetchall()]
            expected_columns = [
                'patient_id', 'first_name', 'surname', 'contact', 'dob', 'gender',
                'medical_aid', 'medical_aid_number', 'occupation',
                'referral_diagnosis', 'physiotherapist_name'
            ]
            if not all(col in columns for col in expected_columns):
                print("PhysioApp: Schema outdated, recreating tables")
                logging.warning("Patients table schema is outdated. Recreating tables.")
                messagebox.showwarning('Warning', 'Database schema is outdated. Recreating tables.')
                self.create_tables()
            else:
                print("PhysioApp: Patients table exists with correct schema")

    def create_tables(self):
        """Create all necessary database tables."""
        print("Creating Patients table")
        self.cursor.execute('DROP TABLE IF EXISTS Patients')
        self.cursor.execute('''
            CREATE TABLE Patients (
                patient_id TEXT PRIMARY KEY,
                first_name TEXT NOT NULL,
                surname TEXT NOT NULL,
                contact TEXT,
                dob DATE,
                gender TEXT,
                medical_aid TEXT,
                medical_aid_number TEXT,
                occupation TEXT,
                referral_diagnosis TEXT NOT NULL,
                physiotherapist_name TEXT NOT NULL
            )
        ''')
        print("Creating MedicalHistory table")
        self.cursor.execute('DROP TABLE IF EXISTS MedicalHistory')
        self.cursor.execute('''
            CREATE TABLE MedicalHistory (
                patient_id TEXT,
                condition TEXT,
                surgery TEXT,
                allergy TEXT,
                FOREIGN KEY (patient_id) REFERENCES Patients (patient_id)
            )
        ''')
        print("Creating Diagnosis table")
        self.cursor.execute('DROP TABLE IF EXISTS Diagnosis')
        self.cursor.execute('''
            CREATE TABLE Diagnosis (
                patient_id TEXT,
                diagnosis TEXT,
                icd_code TEXT,
                FOREIGN KEY (patient_id) REFERENCES Patients (patient_id)
            )
        ''')
        print("Creating TreatmentPlan table")
        self.cursor.execute('DROP TABLE IF EXISTS TreatmentPlan')
        self.cursor.execute('''
            CREATE TABLE TreatmentPlan (
                patient_id TEXT,
                assessment_id TEXT,
                exercise TEXT,
                therapy TEXT,
                modality TEXT,
                interventions TEXT,
                home_exercise_program TEXT,
                FOREIGN KEY (patient_id) REFERENCES Patients (patient_id)
            )
        ''')
        print("Creating Assessments table")
        self.cursor.execute('DROP TABLE IF EXISTS Assessments')
        self.cursor.execute('''
            CREATE TABLE Assessments (
                assessment_id TEXT PRIMARY KEY,
                patient_id TEXT,
                findings TEXT,
                assessment_date DATE,
                subjective_symptoms TEXT,
                objective_findings TEXT,
                history TEXT,
                treatment_notes TEXT,
                FOREIGN KEY (patient_id) REFERENCES Patients (patient_id)
            )
        ''')
        print("Creating Reassessments table")
        self.cursor.execute('DROP TABLE IF EXISTS Reassessments')
        self.cursor.execute('''
            CREATE TABLE Reassessments (
                reassessment_id TEXT PRIMARY KEY,
                patient_id TEXT,
                reassessment_date DATE,
                progress_notes TEXT,
                objective_findings TEXT,
                recommendations TEXT,
                FOREIGN KEY (patient_id) REFERENCES Patients (patient_id)
            )
        ''')
        print("Creating ArchivedPatients table")
        self.cursor.execute('DROP TABLE IF EXISTS ArchivedPatients')
        self.cursor.execute('''
            CREATE TABLE ArchivedPatients (
                archive_id TEXT PRIMARY KEY,
                archive_date DATETIME,
                gender TEXT,
                referral_diagnosis TEXT,
                physiotherapist_name TEXT,
                medical_history_conditions TEXT,
                medical_history_surgeries TEXT,
                medical_history_allergies TEXT,
                diagnosis TEXT,
                icd_code TEXT,
                treatment_exercises TEXT,
                treatment_therapies TEXT,
                treatment_modalities TEXT,
                treatment_interventions TEXT,
                treatment_home_exercise TEXT,
                assessments TEXT,
                reassessments TEXT
            )
        ''')
        self.conn.commit()
        print("All tables created")

    def save_patients(self):
        print("PhysioApp: Saving patients")
        try:
            self.ensure_connection()  # Ensure connection is open
            self.cursor.execute('DELETE FROM Patients')
            self.cursor.execute('DELETE FROM MedicalHistory')
            self.cursor.execute('DELETE FROM Diagnosis')
            self.cursor.execute('DELETE FROM TreatmentPlan')
            self.cursor.execute('DELETE FROM Assessments')
            self.cursor.execute('DELETE FROM Reassessments')

            for patient_id, patient in self.patients.items():
                self.cursor.execute('''
                    INSERT INTO Patients (patient_id, first_name, surname, contact, dob, gender, medical_aid, medical_aid_number,
                                         occupation, referral_diagnosis, physiotherapist_name)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    patient_id, patient.first_name, patient.surname, patient.contact, patient.dob, patient.gender,
                    patient.medical_aid, patient.medical_aid_number, patient.occupation,
                    patient.referral_diagnosis, patient.physiotherapist_name
                ))

                for condition in patient.medical_history.conditions:
                    self.cursor.execute('INSERT INTO MedicalHistory (patient_id, condition) VALUES (?, ?)', (patient_id, condition))
                for surgery in patient.medical_history.surgeries:
                    self.cursor.execute('INSERT INTO MedicalHistory (patient_id, surgery) VALUES (?, ?)', (patient_id, surgery))
                for allergy in patient.medical_history.allergies:
                    self.cursor.execute('INSERT INTO MedicalHistory (patient_id, allergy) VALUES (?, ?)', (patient_id, allergy))

                self.cursor.execute('''
                    INSERT INTO Diagnosis (patient_id, diagnosis, icd_code)
                    VALUES (?, ?, ?)
                ''', (patient_id, patient.diagnosis.diagnosis, patient.diagnosis.icd_code))

                for assessment in patient.assessments:
                    assessment_id = str(uuid.uuid4())
                    self.cursor.execute('''
                        INSERT INTO Assessments (assessment_id, patient_id, findings, assessment_date, subjective_symptoms,
                                                objective_findings, history, treatment_notes)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        assessment_id, patient_id, assessment.findings, assessment.assessment_date,
                        assessment.subjective_symptoms, assessment.objective_findings,
                        assessment.history, assessment.treatment_notes
                    ))
                    # Save treatment plan associated with this assessment
                    if hasattr(assessment, 'treatment_plan') and assessment.treatment_plan:
                        for exercise in assessment.treatment_plan.exercises:
                            self.cursor.execute('INSERT INTO TreatmentPlan (patient_id, assessment_id, exercise) VALUES (?, ?, ?)', (patient_id, assessment_id, exercise))
                        for therapy in assessment.treatment_plan.therapies:
                            self.cursor.execute('INSERT INTO TreatmentPlan (patient_id, assessment_id, therapy) VALUES (?, ?, ?)', (patient_id, assessment_id, therapy))
                        for modality in assessment.treatment_plan.modalities:
                            self.cursor.execute('INSERT INTO TreatmentPlan (patient_id, assessment_id, modality) VALUES (?, ?, ?)', (patient_id, assessment_id, modality))
                        self.cursor.execute('''
                            INSERT INTO TreatmentPlan (patient_id, assessment_id, interventions, home_exercise_program)
                            VALUES (?, ?, ?, ?)
                        ''', (patient_id, assessment_id, assessment.treatment_plan.interventions, assessment.treatment_plan.home_exercise_program))

                for reassessment in patient.reassessments:
                    reassessment_id = str(uuid.uuid4())
                    self.cursor.execute('''
                        INSERT INTO Reassessments (reassessment_id, patient_id, reassessment_date, progress_notes,
                                                  objective_findings, recommendations)
                        VALUES (?, ?, ?, ?, ?, ?)
                    ''', (
                        reassessment_id, patient_id, reassessment.reassessment_date, reassessment.progress_notes,
                        reassessment.objective_findings, reassessment.recommendations
                    ))
                    # Save treatment plan associated with this reassessment
                    if hasattr(reassessment, 'treatment_plan') and reassessment.treatment_plan:
                        for exercise in reassessment.treatment_plan.exercises:
                            self.cursor.execute('INSERT INTO TreatmentPlan (patient_id, assessment_id, exercise) VALUES (?, ?, ?)', (patient_id, reassessment_id, exercise))
                        for therapy in reassessment.treatment_plan.therapies:
                            self.cursor.execute('INSERT INTO TreatmentPlan (patient_id, assessment_id, therapy) VALUES (?, ?, ?)', (patient_id, reassessment_id, therapy))
                        for modality in reassessment.treatment_plan.modalities:
                            self.cursor.execute('INSERT INTO TreatmentPlan (patient_id, assessment_id, modality) VALUES (?, ?, ?)', (patient_id, reassessment_id, modality))
                        self.cursor.execute('''
                            INSERT INTO TreatmentPlan (patient_id, assessment_id, interventions, home_exercise_program)
                            VALUES (?, ?, ?, ?)
                        ''', (patient_id, reassessment_id, reassessment.treatment_plan.interventions, reassessment.treatment_plan.home_exercise_program))

            self.conn.commit()
            print("PhysioApp: Patients saved successfully")

        except Exception as e:
            logging.error(f'Failed to save patients: {str(e)}')
            messagebox.showerror('Error', f'Failed to save patients: {str(e)}')

    def load_patients(self):
        print("PhysioApp: Loading patients")
        try:
            self.ensure_connection()  # Ensure connection is open
            # Verify table schema
            self.cursor.execute("PRAGMA table_info(Patients)")
            columns = [info[1] for info in self.cursor.fetchall()]
            expected_columns = [
                'patient_id', 'first_name', 'surname', 'contact', 'dob', 'gender',
                'medical_aid', 'medical_aid_number', 'occupation',
                'referral_diagnosis', 'physiotherapist_name'
            ]
            if not all(col in columns for col in expected_columns):
                logging.error("Patients table schema is incorrect")
                messagebox.showwarning('Warning', 'Database schema is outdated. Recreating tables.')
                self.create_tables()  # Recreate tables if schema is wrong
                self.patients = {}  # Clear patients dictionary
                self.patient_counter = 1
                return

            self.cursor.execute('SELECT patient_id, first_name, surname, contact, dob, gender, medical_aid, medical_aid_number, '
                              'occupation, referral_diagnosis, physiotherapist_name FROM Patients')
            patients_data = self.cursor.fetchall()

            for patient_data in patients_data:
                patient_id = patient_data[0]
                dob = patient_data[4]  # Already converted by sqlite3 converter

                self.cursor.execute('SELECT condition, surgery, allergy FROM MedicalHistory WHERE patient_id = ?', (patient_id,))
                medical_history_data = self.cursor.fetchall()
                conditions = [row[0] for row in medical_history_data if row[0]]
                surgeries = [row[1] for row in medical_history_data if row[1]]
                allergies = [row[2] for row in medical_history_data if row[2]]

                self.cursor.execute('SELECT diagnosis, icd_code FROM Diagnosis WHERE patient_id = ?', (patient_id,))
                diagnosis_data = self.cursor.fetchone()

                self.cursor.execute('SELECT assessment_id, patient_id, findings, assessment_date, subjective_symptoms, objective_findings, '
                                  'history, treatment_notes FROM Assessments WHERE patient_id = ?', (patient_id,))
                assessments_data = self.cursor.fetchall()

                self.cursor.execute('SELECT reassessment_id, patient_id, reassessment_date, progress_notes, objective_findings, recommendations '
                                  'FROM Reassessments WHERE patient_id = ?', (patient_id,))
                reassessments_data = self.cursor.fetchall()

                medical_history = MedicalHistory(conditions or ['none'], surgeries or ['none'], allergies or ['none'])
                diagnosis = Diagnosis(diagnosis_data[0] if diagnosis_data else 'N/A',
                                    diagnosis_data[1] if diagnosis_data else 'N/A')

                patient = Patient(
                    patient_data[1], patient_data[2], patient_data[3], dob, patient_data[5],
                    patient_data[6], patient_data[7], patient_data[8], patient_data[9],
                    patient_data[10], medical_history, diagnosis
                )

                for assessment_data in assessments_data:
                    assessment_id = assessment_data[0]
                    self.cursor.execute('SELECT exercise, therapy, modality, interventions, home_exercise_program '
                                      'FROM TreatmentPlan WHERE patient_id = ? AND assessment_id = ?', (patient_id, assessment_id))
                    treatment_data = self.cursor.fetchall()
                    exercises = [row[0] for row in treatment_data if row[0]]
                    therapies = [row[1] for row in treatment_data if row[1]]
                    modalities = [row[2] for row in treatment_data if row[2]]
                    interventions = treatment_data[0][3] if treatment_data else 'Initial treatment'
                    home_exercise = treatment_data[0][4] if treatment_data else 'Home exercises prescribed'
                    treatment_plan = TreatmentPlan(exercises or ['none'], therapies or ['none'], modalities or ['none'],
                                                 interventions, home_exercise)
                    assessment = Assessment(
                        assessment_data[2], assessment_data[3], assessment_data[4],
                        assessment_data[5], assessment_data[6], assessment_data[7]
                    )
                    assessment.treatment_plan = treatment_plan
                    patient.add_assessment(assessment)

                for reassessment_data in reassessments_data:
                    reassessment_id = reassessment_data[0]
                    self.cursor.execute('SELECT exercise, therapy, modality, interventions, home_exercise_program '
                                      'FROM TreatmentPlan WHERE patient_id = ? AND assessment_id = ?', (patient_id, reassessment_id))
                    treatment_data = self.cursor.fetchall()
                    exercises = [row[0] for row in treatment_data if row[0]]
                    therapies = [row[1] for row in treatment_data if row[1]]
                    modalities = [row[2] for row in treatment_data if row[2]]
                    interventions = treatment_data[0][3] if treatment_data else 'Initial treatment'
                    home_exercise = treatment_data[0][4] if treatment_data else 'Home exercises prescribed'
                    treatment_plan = TreatmentPlan(exercises or ['none'], therapies or ['none'], modalities or ['none'],
                                                 interventions, home_exercise)
                    reassessment = Reassessment(
                        reassessment_data[2], reassessment_data[3],
                        reassessment_data[4], reassessment_data[5]
                    )
                    reassessment.treatment_plan = treatment_plan
                    patient.add_reassessment(reassessment)

                self.patients[patient_id] = patient
                self.patient_counter = max(self.patient_counter, int(patient_id.split('_')[1]) + 1)

            print("PhysioApp: Patients loaded successfully")

        except Exception as e:
            logging.error(f'Failed to load patients: {str(e)}')
            messagebox.showerror('Error', f'Failed to load patients: {str(e)}')
            self.patients = {}  # Initialize empty patients dictionary to allow GUI to continue
            self.patient_counter = 1

    def safe_call(self, func):
        try:
            func()
        except Exception as e:
            logging.error(f'Error in callback: {str(e)}')
            messagebox.showerror('Error', f'Failed to execute action: {str(e)}')

    def clear_container(self):
        for widget in self.container.winfo_children():
            widget.destroy()

    def bind_scroll_events(self, canvas):
        canvas.bind_all('<MouseWheel>', lambda event: canvas.yview_scroll(int(-1 * (event.delta / 120)), 'units'))
        canvas.bind_all('<Button-4>', lambda event: canvas.yview_scroll(-1, 'units'))
        canvas.bind_all('<Button-5>', lambda event: canvas.yview_scroll(1, 'units'))

    def create_main_menu(self):
        print("PhysioApp: Creating main menu")
        for widget in self.root.winfo_children():
            widget.destroy()
        self.container = tk.Frame(self.root)
        self.container.pack(fill='both', expand=True)
        tk.Button(self.container, text='Patient Details', command=lambda: self.safe_call(self.create_patient_frame), width=20).pack(pady=10)
        tk.Button(self.container, text='Assessment & Reassessment', command=lambda: self.safe_call(self.create_assessment_frame), width=20).pack(pady=10)
        tk.Button(self.container, text='Report', command=lambda: self.safe_call(self.create_report_frame), width=20).pack(pady=10)
        tk.Button(self.container, text='Exit', command=self.root.quit, width=20).pack(pady=10)
        print("PhysioApp: Main menu created")

    def create_patient_frame(self):
        print("PhysioApp: Creating patient frame")
        self.clear_container()
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient='vertical', command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        scrollable_frame.bind(
            '<Configure>',
            lambda e: canvas.configure(scrollregion=canvas.bbox('all'))
        )
        canvas.create_window((0, 0), window=scrollable_frame, anchor='nw')
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        self.frames['patient'] = scrollable_frame
        self.bind_scroll_events(canvas)

        tk.Label(scrollable_frame, text='Select Patient to Edit:').grid(row=0, column=0, padx=5, pady=5, sticky='w')
        self.edit_patient_menu = tk.OptionMenu(scrollable_frame, self.edit_patient_var, *self.get_patient_names_with_details(), command=self.load_patient_details)
        self.edit_patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Select Patient to Delete:').grid(row=1, column=0, padx=5, pady=5, sticky='w')
        self.delete_patient_menu = tk.OptionMenu(scrollable_frame, self.delete_patient_var, *self.get_patient_names_with_details())
        self.delete_patient_menu.grid(row=1, column=1, padx=5, pady=5, sticky='ew')

        tk.Button(scrollable_frame, text='Delete (Duplicate/Error)', command=lambda: self.delete_patient(archive=False)).grid(row=2, column=0, columnspan=2, pady=5)
        tk.Button(scrollable_frame, text='Delete and Archive (Discharge)', command=lambda: self.delete_patient(archive=True)).grid(row=3, column=0, columnspan=2, pady=5)

        fields = [
            ('First Name:', tk.Entry), ('Surname:', tk.Entry), ('Contact:', tk.Entry), ('DOB (YYYY-MM-DD):', tk.Entry),
            ('Gender:', tk.Entry), ('Medical Aid:', tk.Entry), ('Medical Aid Number:', tk.Entry),
            ('Occupation:', tk.Entry), ('Referral Diagnosis:', tk.Entry), ('Physiotherapist Name:', tk.Entry)
        ]
        self.patient_entries = {}
        for i, (label, widget_type) in enumerate(fields):
            tk.Label(scrollable_frame, text=label).grid(row=i+4, column=0, padx=5, pady=5, sticky='w')
            entry = widget_type(scrollable_frame)
            entry.grid(row=i+4, column=1, padx=5, pady=5, sticky='ew')
            self.patient_entries[label.strip(':')] = entry

        tk.Label(scrollable_frame, text='Medical History').grid(row=len(fields)+4, column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text='Conditions (comma-separated):').grid(row=len(fields)+5, column=0, padx=5, pady=5, sticky='w')
        self.conditions_entry = tk.Entry(scrollable_frame)
        self.conditions_entry.grid(row=len(fields)+5, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Surgeries (comma-separated):').grid(row=len(fields)+6, column=0, padx=5, pady=5, sticky='w')
        self.surgeries_entry = tk.Entry(scrollable_frame)
        self.surgeries_entry.grid(row=len(fields)+6, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Allergies (comma-separated):').grid(row=len(fields)+7, column=0, padx=5, pady=5, sticky='w')
        self.allergies_entry = tk.Entry(scrollable_frame)
        self.allergies_entry.grid(row=len(fields)+7, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Diagnosis').grid(row=len(fields)+8, column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text='Diagnosis:').grid(row=len(fields)+9, column=0, padx=5, pady=5, sticky='w')
        self.diagnosis_entry = tk.Entry(scrollable_frame)
        self.diagnosis_entry.grid(row=len(fields)+9, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='ICD Code:').grid(row=len(fields)+10, column=0, padx=5, pady=5, sticky='w')
        self.icd_code_entry = tk.Entry(scrollable_frame)
        self.icd_code_entry.grid(row=len(fields)+10, column=1, padx=5, pady=5, sticky='ew')

        tk.Button(scrollable_frame, text='Add Patient', command=self.add_patient).grid(row=len(fields)+11, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text='Edit Patient', command=self.edit_patient).grid(row=len(fields)+12, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text='Back to Menu', command=self.create_main_menu, width=20).grid(row=len(fields)+13, column=0, columnspan=2, pady=20)

        scrollable_frame.columnconfigure(1, weight=1)
        print("PhysioApp: Patient frame created")

    def create_assessment_frame(self):
        print("PhysioApp: Creating assessment frame")
        self.clear_container()
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient='vertical', command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        scrollable_frame.bind(
            '<Configure>',
            lambda e: canvas.configure(scrollregion=canvas.bbox('all'))
        )
        canvas.create_window((0, 0), window=scrollable_frame, anchor='nw')
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        self.frames['assessment'] = scrollable_frame
        self.bind_scroll_events(canvas)

        tk.Label(scrollable_frame, text='Select Patient:').grid(row=0, column=0, padx=5, pady=5, sticky='w')
        patient_menu = tk.OptionMenu(scrollable_frame, self.edit_patient_var, *self.get_patient_names_with_details(), command=self.load_patient_details)
        patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Assessment Findings:').grid(row=1, column=0, padx=5, pady=5, sticky='w')
        self.assessment_findings_entry = tk.Text(scrollable_frame, height=4, width=40)
        self.assessment_findings_entry.grid(row=1, column=1, padx=5, pady=5, sticky='ew')
        self.assessment_findings_entry.insert('1.0', 'Enter complete sentences describing the assessment findings.')
        self.assessment_findings_entry.bind('<FocusIn>', lambda event: self.clear_placeholder(event, self.assessment_findings_entry, 'Enter complete sentences describing the assessment findings.'))
        self.assessment_findings_entry.bind('<FocusOut>', lambda event: self.restore_placeholder(event, self.assessment_findings_entry, 'Enter complete sentences describing the assessment findings.'))

        tk.Label(scrollable_frame, text='Subjective Symptoms:').grid(row=2, column=0, padx=5, pady=5, sticky='w')
        self.subjective_symptoms_entry = tk.Text(scrollable_frame, height=4, width=40)
        self.subjective_symptoms_entry.grid(row=2, column=1, padx=5, pady=5, sticky='ew')
        self.subjective_symptoms_entry.insert('1.0', 'Enter complete sentences describing the patient\'s reported symptoms.')
        self.subjective_symptoms_entry.bind('<FocusIn>', lambda event: self.clear_placeholder(event, self.subjective_symptoms_entry, 'Enter complete sentences describing the patient\'s reported symptoms.'))
        self.subjective_symptoms_entry.bind('<FocusOut>', lambda event: self.restore_placeholder(event, self.subjective_symptoms_entry, 'Enter complete sentences describing the patient\'s reported symptoms.'))

        tk.Label(scrollable_frame, text='Objective Findings:').grid(row=3, column=0, padx=5, pady=5, sticky='w')
        self.objective_findings_entry = tk.Text(scrollable_frame, height=4, width=40)
        self.objective_findings_entry.grid(row=3, column=1, padx=5, pady=5, sticky='ew')
        self.objective_findings_entry.insert('1.0', 'Enter complete sentences describing the objective findings.')
        self.objective_findings_entry.bind('<FocusIn>', lambda event: self.clear_placeholder(event, self.objective_findings_entry, 'Enter complete sentences describing the objective findings.'))
        self.objective_findings_entry.bind('<FocusOut>', lambda event: self.restore_placeholder(event, self.objective_findings_entry, 'Enter complete sentences describing the objective findings.'))

        tk.Label(scrollable_frame, text='History:').grid(row=4, column=0, padx=5, pady=5, sticky='w')
        self.history_entry = tk.Text(scrollable_frame, height=4, width=40)
        self.history_entry.grid(row=4, column=1, padx=5, pady=5, sticky='ew')
        self.history_entry.insert('1.0', 'Enter the patient\'s relevant medical history or leave as N/A.')
        self.history_entry.bind('<FocusIn>', lambda event: self.clear_placeholder(event, self.history_entry, 'Enter the patient\'s relevant medical history or leave as N/A.'))
        self.history_entry.bind('<FocusOut>', lambda event: self.restore_placeholder(event, self.history_entry, 'Enter the patient\'s relevant medical history or leave as N/A.'))

        tk.Label(scrollable_frame, text='Treatment Notes:').grid(row=5, column=0, padx=5, pady=5, sticky='w')
        self.treatment_notes_entry = tk.Text(scrollable_frame, height=4, width=40)
        self.treatment_notes_entry.grid(row=5, column=1, padx=5, pady=5, sticky='ew')
        self.treatment_notes_entry.insert('1.0', 'Enter complete sentences describing the treatments applied.')
        self.treatment_notes_entry.bind('<FocusIn>', lambda event: self.clear_placeholder(event, self.treatment_notes_entry, 'Enter complete sentences describing the treatments applied.'))
        self.treatment_notes_entry.bind('<FocusOut>', lambda event: self.restore_placeholder(event, self.treatment_notes_entry, 'Enter complete sentences describing the treatments applied.'))

        tk.Label(scrollable_frame, text='Treatment Plan').grid(row=6, column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text='Exercises (comma-separated):').grid(row=7, column=0, padx=5, pady=5, sticky='w')
        self.exercises_entry = tk.Entry(scrollable_frame)
        self.exercises_entry.grid(row=7, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Therapies (comma-separated):').grid(row=8, column=0, padx=5, pady=5, sticky='w')
        self.therapies_entry = tk.Entry(scrollable_frame)
        self.therapies_entry.grid(row=8, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Modalities (comma-separated):').grid(row=9, column=0, padx=5, pady=5, sticky='w')
        self.modalities_entry = tk.Entry(scrollable_frame)
        self.modalities_entry.grid(row=9, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Interventions:').grid(row=10, column=0, padx=5, pady=5, sticky='w')
        self.interventions_entry = tk.Entry(scrollable_frame)
        self.interventions_entry.grid(row=10, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Home Exercise Program:').grid(row=11, column=0, padx=5, pady=5, sticky='w')
        self.home_exercise_entry = tk.Entry(scrollable_frame)
        self.home_exercise_entry.grid(row=11, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Reassessment Progress Notes:').grid(row=12, column=0, padx=5, pady=5, sticky='w')
        self.reassessment_progress_entry = tk.Text(scrollable_frame, height=4, width=40)
        self.reassessment_progress_entry.grid(row=12, column=1, padx=5, pady=5, sticky='ew')
        self.reassessment_progress_entry.insert('1.0', 'Enter complete sentences describing the patient\'s progress.')
        self.reassessment_progress_entry.bind('<FocusIn>', lambda event: self.clear_placeholder(event, self.reassessment_progress_entry, 'Enter complete sentences describing the patient\'s progress.'))
        self.reassessment_progress_entry.bind('<FocusOut>', lambda event: self.restore_placeholder(event, self.reassessment_progress_entry, 'Enter complete sentences describing the patient\'s progress.'))

        tk.Label(scrollable_frame, text='Reassessment Objective Findings:').grid(row=13, column=0, padx=5, pady=5, sticky='w')
        self.reassessment_objective_entry = tk.Text(scrollable_frame, height=4, width=40)
        self.reassessment_objective_entry.grid(row=13, column=1, padx=5, pady=5, sticky='ew')
        self.reassessment_objective_entry.insert('1.0', 'Enter complete sentences describing the reassessment findings.')
        self.reassessment_objective_entry.bind('<FocusIn>', lambda event: self.clear_placeholder(event, self.reassessment_objective_entry, 'Enter complete sentences describing the reassessment findings.'))
        self.reassessment_objective_entry.bind('<FocusOut>', lambda event: self.restore_placeholder(event, self.reassessment_objective_entry, 'Enter complete sentences describing the reassessment findings.'))

        tk.Label(scrollable_frame, text='Recommendations:').grid(row=14, column=0, padx=5, pady=5, sticky='w')
        self.recommendations_entry = tk.Text(scrollable_frame, height=4, width=40)
        self.recommendations_entry.grid(row=14, column=1, padx=5, pady=5, sticky='ew')
        self.recommendations_entry.insert('1.0', 'Enter recommendations for future treatment or leave as N/A.')
        self.recommendations_entry.bind('<FocusIn>', lambda event: self.clear_placeholder(event, self.recommendations_entry, 'Enter recommendations for future treatment or leave as N/A.'))
        self.recommendations_entry.bind('<FocusOut>', lambda event: self.restore_placeholder(event, self.recommendations_entry, 'Enter recommendations for future treatment or leave as N/A.'))

        tk.Button(scrollable_frame, text='Add Assessment', command=self.add_assessment).grid(row=15, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text='Add Reassessment', command=self.add_reassessment).grid(row=16, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text='Back to Menu', command=self.create_main_menu, width=20).grid(row=17, column=0, columnspan=2, pady=20)

        scrollable_frame.columnconfigure(1, weight=1)
        print("PhysioApp: Assessment frame created")

    def create_report_frame(self):
        print("PhysioApp: Creating report frame")
        self.clear_container()
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient='vertical', command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        scrollable_frame.bind(
            '<Configure>',
            lambda e: canvas.configure(scrollregion=canvas.bbox('all'))
        )
        canvas.create_window((0, 0), window=scrollable_frame, anchor='nw')
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        self.frames['report'] = scrollable_frame
        self.bind_scroll_events(canvas)

        tk.Label(scrollable_frame, text='Select Patient for Report:').grid(row=0, column=0, padx=5, pady=5, sticky='w')
        patient_menu = tk.OptionMenu(scrollable_frame, self.edit_patient_var, *self.get_patient_names_with_details(), command=self.load_patient_details)
        patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky='ew')

        tk.Button(scrollable_frame, text='Generate Word Report', command=self.generate_word_report).grid(row=1, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text='Back to Menu', command=self.create_main_menu, width=20).grid(row=2, column=0, columnspan=2, pady=20)

        scrollable_frame.columnconfigure(1, weight=1)
        print("PhysioApp: Report frame created")

    def get_patient_names_with_details(self):
        names = []
        for patient_id, patient in self.patients.items():
            name = f'{patient.first_name} {patient.surname} (ID: {patient_id}, Diagnosis: {patient.referral_diagnosis})'
            names.append(name)
        return names if names else ['No patients available']

    def clear_patient_entries(self):
        if self.frames.get('patient'):
            for entry in self.patient_entries.values():
                entry.delete(0, tk.END)
            self.conditions_entry.delete(0, tk.END)
            self.surgeries_entry.delete(0, tk.END)
            self.allergies_entry.delete(0, tk.END)
            self.diagnosis_entry.delete(0, tk.END)
            self.icd_code_entry.delete(0, tk.END)

    def update_patient_menu(self):
        menu = self.edit_patient_menu['menu']
        menu.delete(0, 'end')
        for name in self.get_patient_names_with_details():
            menu.add_command(label=name, command=lambda value=name: self.edit_patient_var.set(value))
        menu = self.delete_patient_menu['menu']
        menu.delete(0, 'end')
        for name in self.get_patient_names_with_details():
            menu.add_command(label=name, command=lambda value=name: self.delete_patient_var.set(value))

    def add_patient(self):
        print("PhysioApp: Adding patient")
        try:
            self.ensure_connection()  # Ensure connection is open
            first_name = self.patient_entries.get('First Name', tk.Entry(self.frames['patient'])).get()
            surname = self.patient_entries.get('Surname', tk.Entry(self.frames['patient'])).get()
            contact = self.patient_entries.get('Contact', tk.Entry(self.frames['patient'])).get()
            dob_str = self.patient_entries.get('DOB (YYYY-MM-DD)', tk.Entry(self.frames['patient'])).get()
            gender = self.patient_entries.get('Gender', tk.Entry(self.frames['patient'])).get()
            medical_aid = self.patient_entries.get('Medical Aid', tk.Entry(self.frames['patient'])).get()
            medical_aid_number = self.patient_entries.get('Medical Aid Number', tk.Entry(self.frames['patient'])).get()
            occupation = self.patient_entries.get('Occupation', tk.Entry(self.frames['patient'])).get()
            referral_diagnosis = self.patient_entries.get('Referral Diagnosis', tk.Entry(self.frames['patient'])).get()
            physiotherapist_name = self.patient_entries.get('Physiotherapist Name', tk.Entry(self.frames['patient'])).get()

            try:
                dob = datetime.strptime(dob_str, '%Y-%m-%d').date()
            except ValueError:
                messagebox.showerror('Error', 'Invalid DOB format! Use YYYY-MM-DD.')
                return

            self.cursor.execute('''
                SELECT first_name, surname, dob, medical_aid_number FROM Patients 
                WHERE (first_name = ? AND surname = ?) OR dob = ? OR medical_aid_number = ?
            ''', (first_name, surname, dob, medical_aid_number))
            duplicates = self.cursor.fetchall()
            if duplicates:
                duplicate_info = '\n'.join([f'Name: {d[0]} {d[1]}, DOB: {d[2]}, Medical Aid #: {d[3]}' for d in duplicates])
                confirm = messagebox.askyesno(
                    'Potential Duplicate',
                    f'Possible duplicate(s) found:\n{duplicate_info}\n\nProceed with adding new patient?'
                )
                if not confirm:
                    return

            conditions = [c.strip() for c in self.conditions_entry.get().split(',') if c.strip()] or ['none']
            surgeries = [s.strip() for s in self.surgeries_entry.get().split(',') if s.strip()] or ['none']
            allergies = [a.strip() for a in self.allergies_entry.get().split(',') if a.strip()] or ['none']
            diagnosis = self.diagnosis_entry.get() or 'N/A'
            icd_code = self.icd_code_entry.get() or 'N/A'

            if not first_name or not surname or not referral_diagnosis or not physiotherapist_name:
                messagebox.showerror('Error', 'First name, surname, referral diagnosis, and physiotherapist name are required!')
                return

            medical_history = MedicalHistory(conditions, surgeries, allergies)
            diagnosis_obj = Diagnosis(diagnosis, icd_code)

            patient = Patient(
                first_name, surname, contact, dob, gender, medical_aid, medical_aid_number, occupation,
                referral_diagnosis, physiotherapist_name, medical_history, diagnosis_obj
            )

            patient_id = f'patient_{self.patient_counter:03d}'
            self.patients[patient_id] = patient
            self.patient_counter += 1
            self.current_patient = patient
            self.save_patients()
            self.update_patient_menu()
            messagebox.showinfo('Success', f'Patient {first_name} {surname} added!')
            self.clear_patient_entries()
            print("PhysioApp: Patient added successfully")

        except Exception as e:
            logging.error(f'Failed to add patient: {str(e)}')
            messagebox.showerror('Error', f'Failed to add patient: {str(e)}')

    def load_patient_details(self, *args):
        selected = self.edit_patient_var.get()
        if not selected or 'No patients' in selected:
            return
        patient_id = selected.split('ID: ')[1].split(',')[0]
        patient = self.patients.get(patient_id)
        if patient:
            self.current_patient = patient
            # Only update GUI if patient frame is active
            if self.frames.get('patient'):
                self.clear_patient_entries()
                self.patient_entries['First Name'].insert(0, patient.first_name)
                self.patient_entries['Surname'].insert(0, patient.surname)
                self.patient_entries['Contact'].insert(0, patient.contact)
                self.patient_entries['DOB (YYYY-MM-DD)'].insert(0, patient.dob)
                self.patient_entries['Gender'].insert(0, patient.gender)
                self.patient_entries['Medical Aid'].insert(0, patient.medical_aid)
                self.patient_entries['Medical Aid Number'].insert(0, patient.medical_aid_number)
                self.patient_entries['Occupation'].insert(0, patient.occupation)
                self.patient_entries['Referral Diagnosis'].insert(0, patient.referral_diagnosis)
                self.patient_entries['Physiotherapist Name'].insert(0, patient.physiotherapist_name)
                self.conditions_entry.insert(0, ', '.join(patient.medical_history.conditions))
                self.surgeries_entry.insert(0, ', '.join(patient.medical_history.surgeries))
                self.allergies_entry.insert(0, ', '.join(patient.medical_history.allergies))
                self.diagnosis_entry.insert(0, patient.diagnosis.diagnosis)
                self.icd_code_entry.insert(0, patient.diagnosis.icd_code)

    def edit_patient(self):
        if not self.current_patient:
            messagebox.showerror('Error', 'No patient selected!')
            return

        try:
            self.ensure_connection()  # Ensure connection is open
            first_name = self.patient_entries.get('First Name', tk.Entry(self.frames['patient'])).get()
            surname = self.patient_entries.get('Surname', tk.Entry(self.frames['patient'])).get()
            contact = self.patient_entries.get('Contact', tk.Entry(self.frames['patient'])).get()
            dob_str = self.patient_entries.get('DOB (YYYY-MM-DD)', tk.Entry(self.frames['patient'])).get()
            gender = self.patient_entries.get('Gender', tk.Entry(self.frames['patient'])).get()
            medical_aid = self.patient_entries.get('Medical Aid', tk.Entry(self.frames['patient'])).get()
            medical_aid_number = self.patient_entries.get('Medical Aid Number', tk.Entry(self.frames['patient'])).get()
            occupation = self.patient_entries.get('Occupation', tk.Entry(self.frames['patient'])).get()
            referral_diagnosis = self.patient_entries.get('Referral Diagnosis', tk.Entry(self.frames['patient'])).get()
            physiotherapist_name = self.patient_entries.get('Physiotherapist Name', tk.Entry(self.frames['patient'])).get()

            try:
                dob = datetime.strptime(dob_str, '%Y-%m-%d').date()
            except ValueError:
                messagebox.showerror('Error', 'Invalid DOB format! Use YYYY-MM-DD.')
                return

            conditions = [c.strip() for c in self.conditions_entry.get().split(',') if c.strip()] or ['none']
            surgeries = [s.strip() for s in self.surgeries_entry.get().split(',') if s.strip()] or ['none']
            allergies = [a.strip() for s in self.allergies_entry.get().split(',') if s.strip()] or ['none']
            diagnosis = self.diagnosis_entry.get() or 'N/A'
            icd_code = self.icd_code_entry.get() or 'N/A'

            if not first_name or not surname or not referral_diagnosis or not physiotherapist_name:
                messagebox.showerror('Error', 'First name, surname, referral diagnosis, and physiotherapist name are required!')
                return

            medical_history = MedicalHistory(conditions, surgeries, allergies)
            diagnosis_obj = Diagnosis(diagnosis, icd_code)

            patient_id = None
            for pid, p in self.patients.items():
                if p == self.current_patient:
                    patient_id = pid
                    break

            if patient_id:
                self.patients[patient_id] = Patient(
                    first_name, surname, contact, dob, gender, medical_aid, medical_aid_number, occupation,
                    referral_diagnosis, physiotherapist_name, medical_history, diagnosis_obj
                )
                self.patients[patient_id].assessments = self.current_patient.assessments
                self.patients[patient_id].reassessments = self.current_patient.reassessments
                self.current_patient = self.patients[patient_id]
                self.save_patients()
                self.update_patient_menu()
                messagebox.showinfo('Success', f'Patient {first_name} {surname} updated!')
                self.clear_patient_entries()

        except Exception as e:
            logging.error(f'Failed to edit patient: {str(e)}')
            messagebox.showerror('Error', f'Failed to edit patient: {str(e)}')

    def delete_patient(self, archive=False):
        selected = self.delete_patient_var.get()
        if not selected or 'No patients' in selected:
            messagebox.showerror('Error', 'No patient selected!')
            return

        patient_id = selected.split('ID: ')[1].split(',')[0]
        patient = self.patients.get(patient_id)
        if not patient:
            messagebox.showerror('Error', 'Patient not found!')
            return

        confirm = messagebox.askyesno('Confirm Deletion', f'Are you sure you want to delete patient {patient.first_name} {patient.surname}?')
        if not confirm:
            return

        try:
            self.ensure_connection()  # Ensure connection is open
            if archive:
                archive_id = str(uuid.uuid4())
                assessments_str = json.dumps([a.to_dict() for a in patient.assessments])
                reassessments_str = json.dumps([r.to_dict() for r in patient.reassessments])
                # Collect treatment plan data from assessments/reassessments
                all_exercises = []
                all_therapies = []
                all_modalities = []
                all_interventions = []
                all_home_exercises = []
                for assessment in patient.assessments:
                    if hasattr(assessment, 'treatment_plan') and assessment.treatment_plan:
                        all_exercises.extend(assessment.treatment_plan.exercises)
                        all_therapies.extend(assessment.treatment_plan.therapies)
                        all_modalities.extend(assessment.treatment_plan.modalities)
                        all_interventions.append(assessment.treatment_plan.interventions)
                        all_home_exercises.append(assessment.treatment_plan.home_exercise_program)
                for reassessment in patient.reassessments:
                    if hasattr(reassessment, 'treatment_plan') and reassessment.treatment_plan:
                        all_exercises.extend(reassessment.treatment_plan.exercises)
                        all_therapies.extend(reassessment.treatment_plan.therapies)
                        all_modalities.extend(reassessment.treatment_plan.modalities)
                        all_interventions.append(reassessment.treatment_plan.interventions)
                        all_home_exercises.append(reassessment.treatment_plan.home_exercise_program)
                self.cursor.execute('''
                    INSERT INTO ArchivedPatients (
                        archive_id, archive_date, gender, referral_diagnosis, physiotherapist_name,
                        medical_history_conditions, medical_history_surgeries, medical_history_allergies,
                        diagnosis, icd_code, treatment_exercises, treatment_therapies, treatment_modalities,
                        treatment_interventions, treatment_home_exercise, assessments, reassessments
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    archive_id, datetime.now(), patient.gender, patient.referral_diagnosis,
                    patient.physiotherapist_name, json.dumps(patient.medical_history.conditions),
                    json.dumps(patient.medical_history.surgeries), json.dumps(patient.medical_history.allergies),
                    patient.diagnosis.diagnosis, patient.diagnosis.icd_code,
                    json.dumps(list(set(all_exercises))), json.dumps(list(set(all_therapies))),
                    json.dumps(list(set(all_modalities))), json.dumps(list(set(all_interventions))),
                    json.dumps(list(set(all_home_exercises))), assessments_str, reassessments_str
                ))
                self.conn.commit()
                messagebox.showinfo('Success', f'Patient {patient.first_name} {patient.surname} archived!')

            del self.patients[patient_id]
            self.current_patient = None
            self.save_patients()
            self.update_patient_menu()
            self.clear_patient_entries()
            messagebox.showinfo('Success', f'Patient {patient.first_name} {patient.surname} deleted!')

        except Exception as e:
            logging.error(f'Failed to delete patient: {str(e)}')
            messagebox.showerror('Error', f'Failed to delete patient: {str(e)}')

    def clear_placeholder(self, event, widget, placeholder):
        if widget.get('1.0', tk.END).strip() == placeholder:
            widget.delete('1.0', tk.END)

    def restore_placeholder(self, event, widget, placeholder):
        if not widget.get('1.0', tk.END).strip():
            widget.insert('1.0', placeholder)

    def add_assessment(self):
        if not self.current_patient:
            messagebox.showerror('Error', 'No patient selected!')
            return

        try:
            self.ensure_connection()  # Ensure connection is open
            findings = self.assessment_findings_entry.get('1.0', tk.END).strip()
            subjective_symptoms = self.subjective_symptoms_entry.get('1.0', tk.END).strip()
            objective_findings = self.objective_findings_entry.get('1.0', tk.END).strip()
            history = self.history_entry.get('1.0', tk.END).strip()
            treatment_notes = self.treatment_notes_entry.get('1.0', tk.END).strip()
            exercises = [e.strip() for e in self.exercises_entry.get().split(',') if e.strip()] or ['none']
            therapies = [t.strip() for t in self.therapies_entry.get().split(',') if t.strip()] or ['none']
            modalities = [m.strip() for m in self.modalities_entry.get().split(',') if m.strip()] or ['none']
            interventions = self.interventions_entry.get() or 'Initial treatment'
            home_exercise = self.home_exercise_entry.get() or 'Home exercises prescribed'

            if not findings or findings == 'Enter complete sentences describing the assessment findings.':
                messagebox.showerror('Error', 'Assessment findings are required!')
                return

            treatment_plan = TreatmentPlan(exercises, therapies, modalities, interventions, home_exercise)
            assessment = Assessment(
                findings, datetime.now().date(), subjective_symptoms, objective_findings, history, treatment_notes
            )
            assessment.treatment_plan = treatment_plan
            self.current_patient.add_assessment(assessment)
            self.save_patients()
            messagebox.showinfo('Success', 'Assessment added!')
            self.assessment_findings_entry.delete('1.0', tk.END)
            self.subjective_symptoms_entry.delete('1.0', tk.END)
            self.objective_findings_entry.delete('1.0', tk.END)
            self.history_entry.delete('1.0', tk.END)
            self.treatment_notes_entry.delete('1.0', tk.END)
            self.exercises_entry.delete(0, tk.END)
            self.therapies_entry.delete(0, tk.END)
            self.modalities_entry.delete(0, tk.END)
            self.interventions_entry.delete(0, tk.END)
            self.home_exercise_entry.delete(0, tk.END)
            self.restore_placeholder(None, self.assessment_findings_entry, 'Enter complete sentences describing the assessment findings.')
            self.restore_placeholder(None, self.subjective_symptoms_entry, 'Enter complete sentences describing the patient\'s reported symptoms.')
            self.restore_placeholder(None, self.objective_findings_entry, 'Enter complete sentences describing the objective findings.')
            self.restore_placeholder(None, self.history_entry, 'Enter the patient\'s relevant medical history or leave as N/A.')
            self.restore_placeholder(None, self.treatment_notes_entry, 'Enter complete sentences describing the treatments applied.')

        except Exception as e:
            logging.error(f'Failed to add assessment: {str(e)}')
            messagebox.showerror('Error', f'Failed to add assessment: {str(e)}')

    def add_reassessment(self):
        if not self.current_patient:
            messagebox.showerror('Error', 'No patient selected!')
            return

        try:
            self.ensure_connection()  # Ensure connection is open
            progress_notes = self.reassessment_progress_entry.get('1.0', tk.END).strip()
            objective_findings = self.reassessment_objective_entry.get('1.0', tk.END).strip()
            recommendations = self.recommendations_entry.get('1.0', tk.END).strip()
            exercises = [e.strip() for e in self.exercises_entry.get().split(',') if e.strip()] or ['none']
            therapies = [t.strip() for t in self.therapies_entry.get().split(',') if t.strip()] or ['none']
            modalities = [m.strip() for m in self.modalities_entry.get().split(',') if m.strip()] or ['none']
            interventions = self.interventions_entry.get() or 'Initial treatment'
            home_exercise = self.home_exercise_entry.get() or 'Home exercises prescribed'

            if not progress_notes or progress_notes == 'Enter complete sentences describing the patient\'s progress.':
                messagebox.showerror('Error', 'Progress notes are required!')
                return

            treatment_plan = TreatmentPlan(exercises, therapies, modalities, interventions, home_exercise)
            reassessment = Reassessment(
                datetime.now().date(), progress_notes, objective_findings, recommendations
            )
            reassessment.treatment_plan = treatment_plan
            self.current_patient.add_reassessment(reassessment)
            self.save_patients()
            messagebox.showinfo('Success', 'Reassessment added!')
            self.reassessment_progress_entry.delete('1.0', tk.END)
            self.reassessment_objective_entry.delete('1.0', tk.END)
            self.recommendations_entry.delete('1.0', tk.END)
            self.exercises_entry.delete(0, tk.END)
            self.therapies_entry.delete(0, tk.END)
            self.modalities_entry.delete(0, tk.END)
            self.interventions_entry.delete(0, tk.END)
            self.home_exercise_entry.delete(0, tk.END)
            self.restore_placeholder(None, self.reassessment_progress_entry, 'Enter complete sentences describing the patient\'s progress.')
            self.restore_placeholder(None, self.reassessment_objective_entry, 'Enter complete sentences describing the reassessment findings.')
            self.restore_placeholder(None, self.recommendations_entry, 'Enter recommendations for future treatment or leave as N/A.')

        except Exception as e:
            logging.error(f'Failed to add reassessment: {str(e)}')
            messagebox.showerror('Error', f'Failed to add reassessment: {str(e)}')

    def generate_word_report(self):
        if not self.current_patient:
            messagebox.showerror('Error', 'No patient selected!')
            return

        try:
            doc = Document()
            # Add title based on report type
            has_reassessments = len(self.current_patient.reassessments) > 0
            report_type = "Progress" if has_reassessments else "Initial"
            doc.add_heading(f'{report_type} Report: {self.current_patient.first_name} {self.current_patient.surname}', 0)

            # Personal Details
            doc.add_heading('Personal Details', level=1)
            doc.add_paragraph(f'First Name: {self.current_patient.first_name}')
            doc.add_paragraph(f'Surname: {self.current_patient.surname}')
            doc.add_paragraph(f'Contact: {self.current_patient.contact or "N/A"}')
            doc.add_paragraph(f'DOB: {self.current_patient.dob}')
            doc.add_paragraph(f'Gender: {self.current_patient.gender or "N/A"}')
            doc.add_paragraph(f'Medical Aid: {self.current_patient.medical_aid or "N/A"}')
            doc.add_paragraph(f'Medical Aid Number: {self.current_patient.medical_aid_number or "N/A"}')
            doc.add_paragraph(f'Occupation: {self.current_patient.occupation or "N/A"}')
            doc.add_paragraph(f'Referral Diagnosis: {self.current_patient.referral_diagnosis}')
            doc.add_paragraph(f'Physiotherapist: {self.current_patient.physiotherapist_name}')

            # Diagnosis
            doc.add_heading('Diagnosis', level=1)
            doc.add_paragraph(f'Diagnosis: {self.current_patient.diagnosis.diagnosis}')
            doc.add_paragraph(f'ICD Code: {self.current_patient.diagnosis.icd_code}')

            latest_assessment = self.current_patient.assessments[-1] if self.current_patient.assessments else None

            if not latest_assessment:
                messagebox.showerror('Error', 'No assessment available for this patient!')
                return

            # Patient Presentation Paragraph
            presentation = f"{self.current_patient.first_name} {self.current_patient.surname} presented with a referral diagnosis of {self.current_patient.referral_diagnosis}."
            conditions = [c for c in self.current_patient.medical_history.conditions if c != 'none']
            if conditions:
                presentation += f" The patient has a medical history of {', '.join(conditions)}."
            if latest_assessment.subjective_symptoms and latest_assessment.subjective_symptoms != 'Enter complete sentences describing the patient\'s reported symptoms.':
                presentation += f" Reported symptoms include: {latest_assessment.subjective_symptoms}"
            doc.add_paragraph(presentation, style='BodyText')

            # Objective Assessment Paragraph
            objective = "On assessment, it was found that the patient had "
            if latest_assessment.objective_findings and latest_assessment.objective_findings != 'Enter complete sentences describing the objective findings.':
                objective += latest_assessment.objective_findings
            else:
                objective += "no specific objective findings recorded."
            if latest_assessment.findings and latest_assessment.findings != 'Enter complete sentences describing the assessment findings.':
                objective += f" Additional findings include: {latest_assessment.findings}"
            doc.add_paragraph(objective, style='BodyText')

            # Physiotherapy Management and Recommendation Paragraph
            management = "The physiotherapy management plan includes "
            if hasattr(latest_assessment, 'treatment_plan') and latest_assessment.treatment_plan:
                exercises = [e for e in latest_assessment.treatment_plan.exercises if e != 'none']
                therapies = [t for t in latest_assessment.treatment_plan.therapies if t != 'none']
                modalities = [m for m in latest_assessment.treatment_plan.modalities if m != 'none']
                treatment_components = []
                if exercises:
                    treatment_components.append(f"exercises such as {', '.join(exercises)}")
                if therapies:
                    treatment_components.append(f"therapies including {', '.join(therapies)}")
                if modalities:
                    treatment_components.append(f"modalities like {', '.join(modalities)}")
                if latest_assessment.treatment_plan.interventions and latest_assessment.treatment_plan.interventions != 'Initial treatment':
                    treatment_components.append(f"interventions such as {latest_assessment.treatment_plan.interventions}")
                if treatment_components:
                    management += ", ".join(treatment_components) + "."
                else:
                    management += "no specific interventions recorded."
                if latest_assessment.treatment_plan.home_exercise_program and latest_assessment.treatment_plan.home_exercise_program != 'Home exercises prescribed':
                    management += f" A home exercise program was prescribed: {latest_assessment.treatment_plan.home_exercise_program}"
            else:
                management += "no treatment plan recorded."
            if latest_assessment.treatment_notes and latest_assessment.treatment_notes != 'Enter complete sentences describing the treatments applied.':
                management += f" Treatment notes: {latest_assessment.treatment_notes}"
            doc.add_paragraph(management, style='BodyText')

            # Reassessment Paragraph (Progress Report Only)
            if has_reassessments:
                latest_reassessment = self.current_patient.reassessments[-1]
                reassessment = f"On reassessment dated {latest_reassessment.reassessment_date}, the patient's progress was noted as follows: "
                if latest_reassessment.progress_notes and latest_reassessment.progress_notes != 'Enter complete sentences describing the patient\'s progress.':
                    reassessment += latest_reassessment.progress_notes
                else:
                    reassessment += "no progress notes recorded."
                if latest_reassessment.objective_findings and latest_reassessment.objective_findings != 'Enter complete sentences describing the reassessment findings.':
                    reassessment += f" Objective findings include: {latest_reassessment.objective_findings}"
                if hasattr(latest_reassessment, 'treatment_plan') and latest_reassessment.treatment_plan:
                    exercises = [e for e in latest_reassessment.treatment_plan.exercises if e != 'none']
                    therapies = [t for t in latest_reassessment.treatment_plan.therapies if t != 'none']
                    modalities = [m for m in latest_reassessment.treatment_plan.modalities if m != 'none']
                    if exercises or therapies or modalities or latest_reassessment.treatment_plan.interventions != 'Initial treatment':
                        reassessment += " Updated treatment plan includes "
                        treatment_components = []
                        if exercises:
                            treatment_components.append(f"exercises such as {', '.join(exercises)}")
                        if therapies:
                            treatment_components.append(f"therapies including {', '.join(therapies)}")
                        if modalities:
                            treatment_components.append(f"modalities like {', '.join(modalities)}")
                        if latest_reassessment.treatment_plan.interventions and latest_reassessment.treatment_plan.interventions != 'Initial treatment':
                            treatment_components.append(f"interventions such as {latest_reassessment.treatment_plan.interventions}")
                        reassessment += ", ".join(treatment_components) + "."
                    if latest_reassessment.treatment_plan.home_exercise_program and latest_reassessment.treatment_plan.home_exercise_program != 'Home exercises prescribed':
                        reassessment += f" Updated home exercise program: {latest_reassessment.treatment_plan.home_exercise_program}"
                if latest_reassessment.recommendations and latest_reassessment.recommendations != 'Enter recommendations for future treatment or leave as N/A.':
                    reassessment += f" Recommendations: {latest_reassessment.recommendations}"
                doc.add_paragraph(reassessment, style='BodyText')

            # Save the document
            filename = f'{report_type}_Report_{self.current_patient.first_name}_{self.current_patient.surname}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.docx'
            doc.save(filename)
            messagebox.showinfo('Success', f'{report_type} report generated: {filename}')

        except Exception as e:
            logging.error(f'Failed to generate report: {str(e)}')
            messagebox.showerror('Error', f'Failed to generate report: {str(e)}')

print("Cell 3: PhysioApp with Database, GUI, and Patient Management executed")

In [ ]:
#Cell 4 - new
def clear_placeholder(self, event, widget, placeholder):
    if widget.get('1.0', tk.END).strip() == placeholder:
        widget.delete('1.0', tk.END)

def restore_placeholder(self, event, widget, placeholder):
    if not widget.get('1.0', tk.END).strip():
        widget.insert('1.0', placeholder)

def add_assessment(self):
    if not self.current_patient:
        messagebox.showerror('Error', 'No patient selected!')
        return

    try:
        findings = self.assessment_findings_entry.get('1.0', tk.END).strip()
        subjective_symptoms = self.subjective_symptoms_entry.get('1.0', tk.END).strip()
        objective_findings = self.objective_findings_entry.get('1.0', tk.END).strip()
        history = self.history_entry.get('1.0', tk.END).strip()
        treatment_notes = self.treatment_notes_entry.get('1.0', tk.END).strip()

        default_texts = [
            'Enter complete sentences describing the assessment findings.',
            'Enter complete sentences describing the patient\'s reported symptoms.',
            'Enter complete sentences describing the objective findings.',
            'Enter the patient\'s relevant medical history or leave as N/A.',
            'Enter complete sentences describing the treatments applied.'
        ]

        if (findings in default_texts or not findings or
            subjective_symptoms in default_texts or not subjective_symptoms or
            objective_findings in default_texts or not objective_findings):
            messagebox.showerror('Error', 'Findings, subjective symptoms, and objective findings are required!')
            return

        if history in default_texts:
            history = 'N/A'
        if treatment_notes in default_texts:
            treatment_notes = 'N/A'

        assessment = Assessment(
            findings, date.today(), subjective_symptoms,
            objective_findings, history, treatment_notes
        )

        patient_id = None
        for pid, p in self.patients.items():
            if p == self.current_patient:
                patient_id = pid
                break

        if patient_id:
            self.patients[patient_id].add_assessment(assessment)
            self.save_patients()
            messagebox.showinfo('Success', 'Assessment added!')
            self.assessment_findings_entry.delete('1.0', tk.END)
            self.subjective_symptoms_entry.delete('1.0', tk.END)
            self.objective_findings_entry.delete('1.0', tk.END)
            self.history_entry.delete('1.0', tk.END)
            self.treatment_notes_entry.delete('1.0', tk.END)
            self.assessment_findings_entry.insert('1.0', 'Enter complete sentences describing the assessment findings.')
            self.subjective_symptoms_entry.insert('1.0', 'Enter complete sentences describing the patient\'s reported symptoms.')
            self.objective_findings_entry.insert('1.0', 'Enter complete sentences describing the objective findings.')
            self.history_entry.insert('1.0', 'Enter the patient\'s relevant medical history or leave as N/A.')
            self.treatment_notes_entry.insert('1.0', 'Enter complete sentences describing the treatments applied.')

    except Exception as e:
        logging.error(f'Failed to add assessment: {str(e)}')
        messagebox.showerror('Error', f'Failed to add assessment: {str(e)}')

def add_reassessment(self):
    if not self.current_patient:
        messagebox.showerror('Error', 'No patient selected!')
        return

    try:
        progress_notes = self.reassessment_progress_entry.get('1.0', tk.END).strip()
        objective_findings = self.reassessment_objective_entry.get('1.0', tk.END).strip()
        recommendations = self.recommendations_entry.get('1.0', tk.END).strip()

        default_texts = [
            'Enter complete sentences describing the patient\'s progress.',
            'Enter complete sentences describing the reassessment findings.',
            'Enter recommendations for future treatment or leave as N/A.'
        ]

        if (progress_notes in default_texts or not progress_notes or
            objective_findings in default_texts or not objective_findings):
            messagebox.showerror('Error', 'Progress notes and objective findings are required!')
            return

        if recommendations in default_texts:
            recommendations = 'N/A'

        reassessment = Reassessment(
            date.today(), progress_notes, objective_findings, recommendations
        )

        patient_id = None
        for pid, p in self.patients.items():
            if p == self.current_patient:
                patient_id = pid
                break

        if patient_id:
            self.patients[patient_id].add_reassessment(reassessment)
            self.save_patients()
            messagebox.showinfo('Success', 'Reassessment added!')
            self.reassessment_progress_entry.delete('1.0', tk.END)
            self.reassessment_objective_entry.delete('1.0', tk.END)
            self.recommendations_entry.delete('1.0', tk.END)
            self.reassessment_progress_entry.insert('1.0', 'Enter complete sentences describing the patient\'s progress.')
            self.reassessment_objective_entry.insert('1.0', 'Enter complete sentences describing the reassessment findings.')
            self.recommendations_entry.insert('1.0', 'Enter recommendations for future treatment or leave as N/A.')

    except Exception as e:
        logging.error(f'Failed to add reassessment: {str(e)}')
        messagebox.showerror('Error', f'Failed to add reassessment: {str(e)}')

print("Cell 4: Assessment and Reassessment Functions executed")

In [ ]:
#Cell 5 - new

def generate_word_report(self):
    print("PhysioApp: Entering generate_word_report")
    if not self.current_patient:
        messagebox.showerror('Error', 'No patient selected!')
        return

    document = Document()
    document.add_heading(f'Physiotherapy Management Report for {self.current_patient.name}', level=3)
    print("PhysioApp: Document created")

    document.add_paragraph(f'Name: {self.current_patient.name}')
    document.add_paragraph(f'Date of Birth: {self.current_patient.dob}')
    document.add_paragraph(f'Medical Aid: {self.current_patient.medical_aid}')
    document.add_paragraph(f'Medical Aid Number: {self.current_patient.medical_aid_number}')
    document.add_paragraph(f'Occupation: {self.current_patient.occupation}')
    document.add_paragraph(f'Physiotherapist: {self.current_patient.physiotherapist_name}')
    document.add_paragraph(f'Referral Diagnosis: {self.current_patient.referral_diagnosis}')

    document.add_paragraph('Dear Dr,')
    document.add_paragraph(f'Thank you for the referral of the above-mentioned patient.')

    age = (datetime.now().date() - self.current_patient.dob).days // 365
    age_str = f'{age}-year-old' if age >= 1 else f'{(datetime.now().date() - self.current_patient.dob).days // 30}-month-old'

    narrative = f'The patient is a {age_str} {self.current_patient.gender} who presented with {self.current_patient.referral_diagnosis}. '

    if self.current_patient.assessments:
        latest_assessment = self.current_patient.assessments[-1]
        subjective = latest_assessment.subjective_symptoms
        objective = latest_assessment.objective_findings
        history = latest_assessment.history
        findings = latest_assessment.findings
        subjective = subjective if subjective.endswith(('.','?','!')) else subjective + '.'
        objective = objective if objective.endswith(('.','?','!')) else objective + '.'
        history = history if history != 'N/A' and history.endswith(('.','?','!')) else 'no relevant history provided.' if history == 'N/A' else history + '.'
        findings = findings if findings.endswith(('.','?','!')) else findings + '.'
        narrative += (
            f'On assessment, the patient reported {subjective} '
            f'Objective assessment revealed {objective} '
            f'Relevant history includes {history} '
            f'Key findings included {findings} '
        )
    else:
        narrative += 'No assessment has been recorded. '

    interventions = self.current_patient.treatment_plan.interventions
    interventions = interventions if interventions.endswith(('.','?','!')) else interventions + '.'
    narrative += f'Physiotherapy management included {interventions} '

    if self.current_patient.assessments and self.current_patient.assessments[-1].treatment_notes != 'N/A':
        treatment_notes = self.current_patient.assessments[-1].treatment_notes
        treatment_notes = treatment_notes if treatment_notes.endswith(('.','?','!')) else treatment_notes + '.'
        narrative += f'During the session, the following treatments were applied: {treatment_notes} '

    home_exercise = self.current_patient.treatment_plan.home_exercise_program
    home_exercise = home_exercise if home_exercise.endswith(('.','?','!')) else home_exercise + '.'
    narrative += f'A home exercise program was provided, including {home_exercise} '

    if self.current_patient.reassessments:
        latest_reassessment = self.current_patient.reassessments[-1]
        progress = latest_reassessment.progress_notes
        objective = latest_reassessment.objective_findings
        recommendations = latest_reassessment.recommendations
        progress = progress if progress.endswith(('.','?','!')) else progress + '.'
        objective = objective if objective.endswith(('.','?','!')) else objective + '.'
        recommendations = recommendations if recommendations != 'N/A' and recommendations.endswith(('.','?','!')) else 'No specific recommendations provided.' if recommendations == 'N/A' else recommendations + '.'
        narrative += (
            f'On reassessment, the patient reported {progress} '
            f'Objective findings included {objective} '
            f'Recommendations: {recommendations}'
        )
    else:
        narrative += 'No reassessment has been recorded.'

    document.add_paragraph(narrative)
    document.add_paragraph('Kind regards,')
    document.add_paragraph(self.current_patient.physiotherapist_name)

    filename = f'Physiotherapy_Report_{self.current_patient.name}.docx'
    try:
        document.save(filename)
        messagebox.showinfo('Success', f'Report saved as {filename}')
    except Exception as e:
        logging.error(f'Failed to save report: {str(e)}')
        messagebox.showerror('Error', f'Failed to save report: {str(e)}')
    print("PhysioApp: Report generation complete")

print("Cell 5: Report Generation executed")

In [36]:
# Cell 6 - Placeholder no code

In [ ]:
#Cell 7 revised
print("Cell 7: Starting main execution")
if __name__ == '__main__':
    print("Cell 7: Inside if __name__ == '__main__'")
    try:
        print("Cell 7: Initializing PhysioApp")
        app = PhysioApp()
        print("Cell 7: PhysioApp initialized, starting mainloop")
        app.root.mainloop()
        print("Cell 7: Mainloop exited")
        app.close_connection()  # Ensure database connection is closed
    except Exception as e:
        print(f"Cell 7: Error starting application: {str(e)}")
        logging.error(f"Application error: {str(e)}")
    finally:
        print("Cell 7: Main execution complete")

In [ ]:
# Cell 8 - Test Script
app = PhysioApp()
try:
    # Create patient frame to initialize patient_entries
    app.create_patient_frame()
    # Simulate adding a patient
    app.patient_entries['First Name'].insert(0, "Jane")
    app.patient_entries['Surname'].insert(0, "Doe")
    app.patient_entries['DOB (YYYY-MM-DD)'].insert(0, "1985-05-10")
    app.patient_entries['Gender'].insert(0, "Female")
    app.patient_entries['Referral Diagnosis'].insert(0, "Knee pain")
    app.patient_entries['Physiotherapist Name'].insert(0, "Dr. Lee")
    app.conditions_entry.insert(0, "Arthritis")
    app.diagnosis_entry.insert(0, "Osteoarthritis")
    app.icd_code_entry.insert(0, "M17.9")
    app.exercises_entry.insert(0, "Quadriceps strengthening")
    app.add_patient()
    print("Test patient added")
except Exception as e:
    print(f"Test failed: {str(e)}")
app.close_connection()